In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer, CrossEncoder, util
from IPython.display import display

/home/fgv/repos/TCC/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
reports = {}
reports["easy"] = pd.read_csv("../reports/temp01_ctxt7000/results_easy_v1.csv")
reports["medium"] = pd.read_csv("../reports/temp01_ctxt7000/results_medium_v1.csv")
reports["hard"] = pd.read_csv("../reports/temp01_ctxt7000/results_hard_v1.csv")

models = ['mistral:7b', 'mistral:7b-q8', 'mistral:7b-fp16']

In [ ]:
for level, df in reports.items():
    print("Tamanho médio de resposta da dificuldade", level, ":")
    for model in models:
        print(df[f'{model}_answer'].str.len().mean())

In [3]:
for level, df in reports.items():
    print(f"\nAnálise para o nível de dificuldade: {level}\n")
    
    print(f"Média de tempo do modelo Q4: {df['mistral:7b_time'].mean():.2f} segundos")
    print(f"Média de tempo do modelo Q8: {df['mistral:7b-q8_time'].mean():.2f} segundos")
    print(f"Média de tempo do modelo FP16: {df['mistral:7b-fp16_time'].mean():.2f} segundos")
    


Análise para o nível de dificuldade: easy

Média de tempo do modelo Q4: 44.41 segundos
Média de tempo do modelo Q8: 49.93 segundos
Média de tempo do modelo FP16: 68.16 segundos

Análise para o nível de dificuldade: medium

Média de tempo do modelo Q4: 45.51 segundos
Média de tempo do modelo Q8: 44.60 segundos
Média de tempo do modelo FP16: 63.02 segundos

Análise para o nível de dificuldade: hard

Média de tempo do modelo Q4: 45.67 segundos
Média de tempo do modelo Q8: 53.65 segundos
Média de tempo do modelo FP16: 304.10 segundos


In [4]:
import numpy as np
# Função Sigmoide: 1 / (1 + e^-x)
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
bi_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

for level, df in reports.items():
    for model in models:
        # Preparar pares para o modelo [(ideal, modelo), (ideal, modelo)...]
        pairs = list(zip(df['resposta'], df[f'{model}_answer']))

        # Gerar embeddings
        embeddings1 = bi_model.encode(df['resposta'].tolist(), convert_to_tensor=True)
        embeddings2 = bi_model.encode(df[f'{model}_answer'].tolist(), convert_to_tensor=True)

        # Calcular similaridade de cosseno
        cosine_scores = util.cos_sim(embeddings1, embeddings2)

        # Aplica pequena penalidade para respostas muito curtas ou muito longas
        for i in range(len(df)):
            len_diff = abs(len(df['resposta'][i].split()) - len(df[f'{model}_answer'][i].split()))
            penalty = sigmoid(len_diff / 10) * 0.1  # Penalidade entre 0 e 0.1
            cosine_scores[i] -= penalty

        # Pegamos a diagonal (comparação item a item correspondente)
        df[f'{model}_score'] = [cosine_scores[i][i].item() for i in range(len(df))]

        print(f"\nAnálise de similaridade para o nível de dificuldade: {level}, modelo: {model}")
        print(f"Média do score de similaridade: {df[f'{model}_score'].mean():.4f}")


Análise de similaridade para o nível de dificuldade: easy, modelo: mistral:7b
Média do score de similaridade: 0.6899

Análise de similaridade para o nível de dificuldade: easy, modelo: mistral:7b-q8
Média do score de similaridade: 0.6896

Análise de similaridade para o nível de dificuldade: easy, modelo: mistral:7b-fp16
Média do score de similaridade: 0.6923

Análise de similaridade para o nível de dificuldade: medium, modelo: mistral:7b
Média do score de similaridade: 0.6749

Análise de similaridade para o nível de dificuldade: medium, modelo: mistral:7b-q8
Média do score de similaridade: 0.6735

Análise de similaridade para o nível de dificuldade: medium, modelo: mistral:7b-fp16
Média do score de similaridade: 0.6745

Análise de similaridade para o nível de dificuldade: hard, modelo: mistral:7b
Média do score de similaridade: 0.6673

Análise de similaridade para o nível de dificuldade: hard, modelo: mistral:7b-q8
Média do score de similaridade: 0.6518

Análise de similaridade para o